In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from utils_prodiac.mongo import Mongo
import matplotlib.pyplot as plt
pd.plotting.register_matplotlib_converters()

In [2]:
pd.set_option('precision', 4)
pd.set_option('display.max_rows', 100)

In [3]:
mongo = Mongo()

# Preparar datos

In [4]:
id_measurement = 1
df_original = mongo.mongo_to_df("twitter",{"id_measurement":id_measurement})

df_original.drop(columns=["data"])
df_original.to_csv("cluster.csv")

In [5]:
#df_original.describe()

In [6]:
#limpieza basica y crear variables para clusterizar
try:
    df_original.drop(columns=["_id"],inplace=True)
except:
    pass

df_original_columns = list(df_original.columns) 
exclude_dates = []

for column in df_original_columns:
    
    if df_original[column].dtype == "int" or df_original[column].dtype == "float":
        df_original[column].fillna(0, inplace = True)
        df_original[column + '_to_cluster'] = df_original[column]
    elif df_original[column].dtype == (object):
        df_original[column].fillna("Sin dato", inplace = True)
        df_original[column + '_to_cluster'] = df_original[column]
    elif df_original[column].dtype == "bool":
        df_original[column].fillna(False, inplace = True)
        df_original[column + '_to_cluster'] = df_original[column]
    else :
        df_original[column+'_year'] = df_original[column].dt.year 
        df_original[column+'_month'] = df_original[column].dt.month 
        df_original[column+'_dayofweek'] = df_original[column].dt.dayofweek
        df_original[column+'_hour'] = df_original[column].dt.hour 
        df_original[column+'_year_to_cluster'] = df_original[column].dt.year 
        df_original[column+'_hour_to_cluster'] = df_original[column].dt.hour 
        df_original[column+'_month_to_cluster'] = df_original[column].dt.month 
        df_original[column+'_dayofweek_to_cluster'] = df_original[column].dt.dayofweek 
        exclude_dates.append(column)

In [7]:
# Label Encoder
from sklearn.preprocessing import LabelEncoder

for column in df_original_columns:
    if df_original[column].dtype == (object):
        encoder = LabelEncoder()
        try:
            df_original[column + '_to_cluster'] = encoder.fit_transform(df_original[column + '_to_cluster'])
        except TypeError as t_err:
            try:
                df_original.drop(columns=[column + '_to_cluster'],inplace=True)
            except KeyError as k_err:
                print("Ignorado " + column+" - KeyError: "+str(k_err))
            print("Ignorado " + column+" - TypeError: "+str(t_err))


Ignorado targets - TypeError: argument must be a string or number
Ignorado data - TypeError: argument must be a string or number


In [8]:
#Feature selector that removes all low-variance features.

df_colums_selected = []

for column in df_original_columns:
    if column not in exclude_dates:
        try:
            varianza = df_original[column + '_to_cluster'].var()
        except KeyError:
            varianza = 0
        
        if  varianza > 0:
            df_colums_selected.append(column)
        else:
            print ("Variables exluidas: " + column)

df_colums_selected = [col+"_to_cluster" for col in df_colums_selected]

Variables exluidas: geo
Variables exluidas: coordinates
Variables exluidas: retweet_count
Variables exluidas: favorite_count
Variables exluidas: reply_count
Variables exluidas: favorited
Variables exluidas: retweeted
Variables exluidas: filter_level
Variables exluidas: lang
Variables exluidas: user_lang
Variables exluidas: targets
Variables exluidas: id_measurement
Variables exluidas: data


In [9]:
# segmentacion por Kmeans

from sklearn.cluster import *
from sklearn import metrics
import json
results = []
if True:
    number_clusters = range(2,9)
    key = "KMeans"
    for k_clusters in number_clusters:
        print ("____________________________________")
        print (" Modelo : %s" % key)
        model = MiniBatchKMeans(n_clusters=k_clusters, n_init=200, random_state=0)
        cluster_fit = model.fit(df_original[df_colums_selected])
        predict = cluster_fit.predict(df_original[df_colums_selected])
        df_y = pd.DataFrame(predict)
        df_original["TEMP_CLUSTER_"+str(k_clusters)] = pd.Series(predict, index=df_original.index)
        df_original["TEMP_CLUSTER_"+str(k_clusters)] = df_original["TEMP_CLUSTER_"+str(k_clusters)] + 1
        table_df = pd.DataFrame(df_y.groupby(0).size())[0]
        table = json.loads(table_df.to_json(orient='columns'))
        print(table_df)
        #table_df.plot.pie(y='"KMEDIAS_CLUSTER_"+str(k_clusters)', figsize=(5, 5))

            # The Silhouette Coefficient is calculated using the mean intra-cluster distance (a) 
            # and the mean nearest-cluster distance (b) for each sample. 
            # The Silhouette Coefficient for a sample is (b - a) / max(a, b). 
            # To clarify, b is the distance between a sample and the nearest cluster that the sample is not a part of. 
            # Note that Silhouette Coefficient is only defined if number of labels is 2 <= n_labels <= n_samples - 1.
            # This function returns the mean Silhouette Coefficient over all samples. 
            # The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters.
            # Negative values generally indicate that a sample has been assigned to the wrong cluster, 
            # as a different cluster is more similar.
        sample_size=int(len(df_y)/2)

        silhouette=metrics.silhouette_score(df_original[df_colums_selected], predict,sample_size=sample_size ,random_state=0)
        print("Silhouette Coefficient: %0.3f" % silhouette )

            # The score is defined as the average similarity measure of each cluster with its most similar cluster, 
            # where similarity is the ratio of within-cluster distances to between-cluster distances. 
            # Thus, clusters which are farther apart and less dispersed will result in a better score
            # The minimum score is zero, with lower values indicating better clustering.
        davies_bouldin_score =  metrics.davies_bouldin_score(df_original[df_colums_selected], predict)
        print("Davies-Bouldin score: %0.3f" % davies_bouldin_score)

            # The score is defined as ratio between the within-cluster dispersion and the between-cluster dispersion.
            # There is no "acceptable" cut-off value. You simply compare CH values by eye. The higher the value, 
            # the "better" is the solution. If on the line-plot of CH values there appears that one solution give 
            # a peak or at least an abrupt elbow, choose it. If, on the contrary, the line is smooth - horizontal
            # or ascending or descending - then there is no reason to prefer one solution to others.
        calinski_harabasz_score = metrics.calinski_harabasz_score(df_original[df_colums_selected], predict)
        print("Calinski and Harabasz score: %0.3f" % calinski_harabasz_score)
        
        #print(model.cluster_centers_)
  
        for k,v in table.items():
            results.append( {
                            "method": key,
                            "n_clusters":k_clusters,
                            "silhouette_score":silhouette,
                            "davies_bouldin_score":davies_bouldin_score,
                            "calinski_harabasz_score":calinski_harabasz_score,
                            "cluster":k,
                            "num_columns":v,
                            #"centroids":model.cluster_centers_,
                           })




____________________________________
 Modelo : KMeans
0
0    11632
1     7980
Name: 0, dtype: int64
Silhouette Coefficient: 0.407
Davies-Bouldin score: 1.236
Calinski and Harabasz score: 9106.793
____________________________________
 Modelo : KMeans
0
0    9424
1    7986
2    2202
Name: 0, dtype: int64
Silhouette Coefficient: 0.471
Davies-Bouldin score: 1.112
Calinski and Harabasz score: 9055.161
____________________________________
 Modelo : KMeans
0
0    5671
1    6491
2    2202
3    5248
Name: 0, dtype: int64
Silhouette Coefficient: 0.553
Davies-Bouldin score: 0.907
Calinski and Harabasz score: 10337.847
____________________________________
 Modelo : KMeans
0
0    1348
1    5602
2    4976
3    5177
4    2509
Name: 0, dtype: int64
Silhouette Coefficient: 0.627
Davies-Bouldin score: 0.844
Calinski and Harabasz score: 13087.503
____________________________________
 Modelo : KMeans
0
0    5602
1    4947
2    2565
3    2445
4    2705
5    1348
Name: 0, dtype: int64
Silhouette Coefficient

In [10]:
#Guardamos resultados en mongo
df_results = pd.DataFrame(results)
mongo.delete_many("cluster_results",{"id_measurement":id_measurement,"method":key})
mongo.df_to_mongo(df_results,"cluster_results")

In [11]:
#Listamos los mejores resultados
df_clusters_selected = df_results.loc[df_results['silhouette_score'] >= df_results.silhouette_score.max()-0.05]
df_clusters_selected= df_clusters_selected.groupby(["n_clusters"] ).first() 
df_clusters_selected.head(100)

,method,silhouette_score,davies_bouldin_score,calinski_harabasz_score,cluster,num_columns
n_clusters,,,,,,
6,KMeans,0.7246,0.7629,16570.1256,0,5602
7,KMeans,0.7560,0.8328,20126.9536,0,2641
8,KMeans,0.7644,0.9231,20437.7866,0,2605


In [12]:
#Seleccionamos el mejor resultado
max_calinski_harabasz_score=df_clusters_selected["calinski_harabasz_score"].max()
df_cluster_selected = df_results.loc[df_results['calinski_harabasz_score'] == max_calinski_harabasz_score]
df_original["Cluster Kmeans"]=df_original["TEMP_CLUSTER_"+ str(int(df_cluster_selected["n_clusters"].mean()))]
df_original.drop( columns=df_original.filter(regex='TEMP_CLUSTER_').columns,inplace= True)
print ("Cluster seleccionado: ",df_cluster_selected["n_clusters"].mean())

Cluster seleccionado:  8.0


In [13]:
#df_original.plot.pie(y="Cluster", figsize=(5, 5))

In [14]:
#borrar variables transformadas

df_columns_to_drop = df_original.filter(regex='_to_cluster')
df_columns_to_drop = df_columns_to_drop.columns
df_original.drop(columns=df_columns_to_drop,inplace=True)
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19612 entries, 0 to 19611
Data columns (total 46 columns):
id_tweet                    19612 non-null int64
text_tweet                  19612 non-null object
date                        19612 non-null datetime64[ns]
is_retweet                  19612 non-null bool
retweet_status_id           19612 non-null float64
retweet_text                19612 non-null object
retweet_user_id             19612 non-null float64
retweet_user_screen_name    19612 non-null object
geo                         19612 non-null object
coordinates                 19612 non-null object
retweet_count               19612 non-null int64
favorite_count              19612 non-null int64
reply_count                 19612 non-null int64
favorited                   19612 non-null bool
retweeted                   19612 non-null bool
filter_level                19612 non-null object
lang                        19612 non-null object
is_quote_status             19612 non-nul

In [15]:
#guardamos y creamos dataframe final
mongo.delete_many("data_transform",{"id_measurement":id_measurement})
mongo.df_to_mongo(df_original,"data_transform")

In [16]:
df_original.drop(columns=["data",], inplace=True)
df_original.to_csv("cluster.csv", date_format='%Y%m%d')